# Compilation of ROMS


The Regional Ocean Modeling System (ROMS) is a free-surface, terrain-following,
primitive equations ocean model widely used by the scientific community for
a diverse range of applications.

## Getting the source code

The source code of ROMS is distributed via [GitHub](https://github.com/myroms/roms).
We use version 4.1 of the ROMS code.
The code is downloaded to the directory `~/src/roms`:

In [ ]:
using ROMS

romsdir = expanduser("~/src/roms")
if !isdir(romsdir)
    mkpath(dirname(romsdir))
    cd(dirname(romsdir)) do
        run(`git clone https://github.com/myroms/roms`)
        cd("roms") do
            run(`git checkout roms-4.1`)
        end
    end
end

The previous julia commands, are essentially the same as the following shell commands:
```bash
mkdir ~/src/
cd ~/src/
git clone https://github.com/myroms/roms
cd roms
git checkout roms-4.1
```

The output of the last command will tell you that `You are in 'detached HEAD' state.` (this is not an error).

All files that are specific to a given implementation of ROMS will be
saved in a different directory `modeldir`:

In [ ]:
modeldir = expanduser("~/ROMS-implementation-test")
mkpath(modeldir);

## The header file

Before we can compile ROMS we need to
* activate diffent terms of the momentum equations
* specify the schemes uses for advection, horizontal mixing,
  type equation of state, ...

The header file controls the compilation of the ROMS model by telling the
compiler which part of the code needs to be compiled. If you modify this file,
ROMS need to be recompiled.

This header file should be named `yourdomain.h` (e.g. `liguriansea.h` for the Ligurian Sea)
and created in the directory `ROMS-implementation-test`.

Do not change the two first lines and the last line of the following cell.
When you execute the cell, the header file with the specified content is
created.

In [ ]:
header_file = joinpath(modeldir,"liguriansea.h")
write(header_file,"""
#define UV_ADV                    /* turn ON advection terms */
#define UV_COR                    /* turn ON Coriolis term */
#define DJ_GRADPS                 /* Splines density  Jacobian (Shchepetkin, 2000) */
#define NONLIN_EOS                /* define if using nonlinear equation of state */
#define SALINITY                  /* define if using salinity */
#define SOLVE3D                   /* define if solving 3D primitive equations */
#define MASKING                   /* define if there is land in the domain */
#define TS_U3HADVECTION           /* Third-order upstream horizontal advection of tracers */
#define TS_C4VADVECTION           /* Fourth-order centered vertical advection of tracers */
#define TS_DIF2                   /* use to turn ON or OFF harmonic horizontal mixing  */
#define MIX_S_UV                  /* mixing along constant S-surfaces */
#define UV_VIS2                   /* turn ON Laplacian horizontal mixing */
#define AVERAGES
#define UV_QDRAG
#define MIX_S_TS

#define  MY25_MIXING
#ifdef MY25_MIXING
#define N2S2_HORAVG
#define KANTHA_CLAYSON
#endif

#define ANA_BSFLUX                /* analytical bottom salinity flux */
#define ANA_BTFLUX                /* analytical bottom temperature flux */
#define ANA_SSFLUX

#define BULK_FLUXES               /* turn ON bulk fluxes computation */
#define CLOUDS
#define LONGWAVE
#define SOLAR_SOURCE
""");

The [ROMS wiki](https://www.myroms.org/wiki/Documentation_Portal) give more information about the [compiler different options](https://www.myroms.org/wiki/Options).

## Compiling the model code

ROMS can use the MPI ([Message Passing Interface](https://en.wikipedia.org/wiki/Message_Passing_Interface)) or OpenMP ([Open Multi-Processing](https://en.wikipedia.org/wiki/OpenMP)) for parallelization (but not both at the same time):

In [ ]:
use_mpi = false;
use_openmp = true;
# or
##use_mpi = true;
##use_openmp = false;

ROMS can either be build (i.e. compiled) the shell script `build_roms.sh` or
with the julia script `ROMS.build`.

`roms_application` is a descriptive name of the domain or the particular application
that the use can choose. We compile ROMS with the [GNU Fortran](https://en.wikipedia.org/wiki/GNU_Fortran) compiler using 8 jobs for compilation.

In [ ]:
roms_application = "LigurianSea"
fortran_compiler = "gfortran"
jobs = 2
logfile = "roms_build.log"

ROMS.build(romsdir,roms_application,modeldir;
           stdout = logfile,
           jobs,
           fortran_compiler,
           use_openmp,
           use_mpi)

The first and last 5 lines of this log file:

In [ ]:
println.(collect(eachline(logfile))[1:5]);
println("...")
println.(collect(eachline(logfile))[end-5:end]);

# Preparation of the input files for ROMS


ROMS needs several input files in the NetCDF format, in paricular:

* the model grid
* the initial conditions
* the boundary conditions
* the atmospheric forcing fields

Optionally
* the climatology file
* the field defining the nudging strength


This script can use multiple threads if [julia was started with multi-threading](https://docs.julialang.org/en/v1/manual/multi-threading/)
(option `-t`/`--threads` or the environement variable `JULIA_NUM_THREADS`)

In [ ]:
using Dates
using ROMS
using ROMS: whenopen
using Downloads: download

## The model bathymetry

While the full [GEBCO bathymetry](https://dox.ulg.ac.be/index.php/s/iEh7ompNdj8AN2p/download)
is relatively large, where use here a subset of the global bathymetry to
reduce the downloading time.
(longitude from 5°E to 15°E and latitude from 40°N to 45°N)

In [ ]:
bath_name = expanduser("~/Data/Bathymetry/gebco_30sec_1_ligurian_sea.nc")

if !isfile(bath_name)
    mkpath(dirname(bath_name))
    download("https://dox.ulg.ac.be/index.php/s/piwSaFP3nhM8jSD/download",bath_name)
end;

The time range for the simulation:
* `t0` start time
* `t1` end time

In [ ]:
t0 = DateTime(2023,1,1);
t1 = DateTime(2023,1,4);

Define the bounding box the of the grid

In [ ]:
# range of longitude
xr = [7.6, 12.2];

# range of latitude
yr = [42, 44.5];

# reduce bathymetry in x and y direction
red = (4, 4)

# maximum normalized topographic variations
rmax = 0.4;

# minimal depth
hmin = 2; # m

# name of folders and files
modeldir = expanduser("~/ROMS-implementation-test")

# The model grid (`GRDNAME` in roms.in)
grd_name = joinpath(modeldir,"roms_grd_liguriansea.nc");

Some parameters specific to the vertical coordinate system

In [ ]:
opt = (
    Tcline = 50,   # m
    theta_s = 5,   # surface refinement
    theta_b = 0.4, # bottom refinement
    nlevels = 32,  # number of vertical levels
    Vtransform  = 2,
    Vstretching = 4,
)

Create the model directory and generate the model grid

In [ ]:
mkpath(modeldir);

domain = ROMS.generate_grid(grd_name,bath_name,xr,yr,red,opt,hmin,rmax);

@info "domain size $(size(domain.mask))"

## The boundary and initial conditions

In [ ]:
# GCM interpolated on model grid (`CLMNAME` in roms.in)
clm_name =  joinpath(modeldir,"roms_clm_2023.nc")

# initial conditions (`ININAME` in roms.in)
ini_name =  joinpath(modeldir,"roms_ini_2023.nc")

# boundary conditions (`BRYNAME` in roms.in)
bry_name =  joinpath(modeldir,"roms_bry_2023.nc")

# temporary directory of the OGCM data
outdir = joinpath(modeldir,"OGCM")
mkpath(outdir)

* For CMEMS boundary conditions [https://marine.copernicus.eu/](https://marine.copernicus.eu/):
   * You may need to adapt the CMEMS `product_id` and `mapping` (if the model domain is outside of the Mediterranean Sea)
   * Data will be downloaded and saved in NetCDF by "chunks" of 60 days in the folder `OGCM` under the content of the variable `basedir`
   * You need to remove the files in this directory if you rerun the script with a different time range.

Here we use the following dataset:
[https://doi.org/10.25423/CMCC/MEDSEA_MULTIYEAR_PHY_006_004_E3R1](https://doi.org/10.25423/CMCC/MEDSEA_MULTIYEAR_PHY_006_004_E3R1)

In [ ]:
product_id = "MEDSEA_MULTIYEAR_PHY_006_004"

mapping the variable (CF names) with the CMEMS `dataset_id`

In [ ]:
mapping = Dict(
    :sea_surface_height_above_geoid => "med-cmcc-ssh-rean-d",
    :sea_water_potential_temperature => "med-cmcc-tem-rean-d",
    :sea_water_salinity => "med-cmcc-sal-rean-d",
    :eastward_sea_water_velocity => "med-cmcc-cur-rean-d",
    :northward_sea_water_velocity => "med-cmcc-cur-rean-d",
)

dataset = ROMS.CMEMS_zarr(product_id,mapping,outdir, time_shift = 12*60*60)

Extent the time range by one extra day

In [ ]:
ROMS.interp_clim(domain,clm_name,dataset,[t0-Dates.Day(1), t1+Dates.Day(1)])

ROMS.extract_ic(domain,clm_name,ini_name, t0);
ROMS.extract_bc(domain,clm_name,bry_name)

Nudging coefficients (`NUDNAME`)

In [ ]:
tscale = 7; # days
alpha = 0.3;
halo = 2;
Niter = 50
max_tscale = 5e5

nud_name = joinpath(modeldir,"roms_nud_$(tscale)_$(Niter).nc")
tracer_NudgeCoef = ROMS.nudgecoef(domain,nud_name,alpha,Niter,
          halo,tscale; max_tscale = max_tscale);

## The atmospheric forcings

Prepare atmospheric forcings (`FRCNAME`)

In [ ]:
ecmwf_fname = expanduser("~/Data/Atmosphere/ecmwf_operational_archive_2022-12-01_2024-02-01.nc")

if !isfile(ecmwf_fname)
    mkpath(dirname(ecmwf_fname))
    download("https://data-assimilation.net/upload/OCEA0036/ecmwf_operational_archive_2022-12-01_2024-02-01.nc",ecmwf_fname)
end

frc_name_prefix = joinpath(modeldir,"roms_frc_2023_")
domain_name = "Ligurian Sea Region"
Vnames = ["sustr","svstr","shflux","swflux","swrad","Uwind","Vwind",
    "lwrad","lwrad_down","latent","sensible","cloud","rain","Pair","Tair","Qair"]

# forcing_filenames corresponds to `FRCNAME` in roms.in
forcing_filenames = ROMS.prepare_ecmwf(ecmwf_fname,Vnames,frc_name_prefix,domain_name)

We print a list of all generated files.

In [ ]:
fn(name) = basename(name) # use relative file path
# fn(name) = name         # use absolute file path

println()
println("The created netCDF files are in $modeldir.");
println("The following information has to be added to roms.in. A template of this file is")
println("provided in the directory User/External of your ROMS source code")
println("You can also use relative or absolute file names.")
println()
println("! grid file ")
println("     GRDNAME == $(fn(grd_name))")
println()
println("! initial conditions")
println("     ININAME == $(fn(ini_name))")
println()
println("! boundary conditions")
println("     NBCFILES == 1")
println("     BRYNAME == $(fn(bry_name))")
println()
println("! climatology or large-scale circulatio model")
println("     NCLMFILES == 1")
println("     CLMNAME == $(fn(clm_name))")
println()
println("! nudging coefficients file (optional)")
println("     NUDNAME == $(fn(nud_name))")
println()
println("! forcing files")
println("     NFFILES == $(length(Vnames))")

for i in 1:length(Vnames)
    if i == 1
        print("     FRCNAME == ")
    else
        print("                ")
    end
    print("$(fn(frc_name_prefix))$(Vnames[i]).nc")
    if i < length(Vnames)
        print(" \\")
    end
    println()
end

Check the resulting files such as bathymetry, initial conditions,
boundary conditions, interpolated model (`clm_name` file) and visualizing them.

## Configuration files

Beside the created NetCDF files, ROMS needs two configuration files
(`roms.in` and `varinfo.yaml`)

In [ ]:
romsdir = expanduser("~/src/roms")
modeldir = expanduser("~/ROMS-implementation-test")
simulationdir = joinpath(modeldir,"Simulation1")
mkpath(simulationdir)

frc_name = joinpath.(modeldir,sort(filter(startswith("roms_frc"),readdir(modeldir))));

Copy `varinfo.yaml` from `~/src/roms/ROMS/External/varinfo.yaml` in your
directory for your simulation (e.g. `ROMS-implementation-test`).
This file does not need to be changed.

In [ ]:
var_name_template = joinpath(romsdir,"ROMS","External","varinfo.yaml")
var_name = joinpath(simulationdir,"varinfo.yaml")
cp(var_name_template,var_name; force=true);

Load the ROMS grid

In [ ]:
domain = ROMS.Grid(grd_name);

We use `roms.in` from `~/src/roms/User/External/roms.in` as a template

In [ ]:
intemplate = joinpath(romsdir,"User","External","roms.in")
infile = joinpath(simulationdir,"roms.in");

This file is typicall edited with a text editor (when editing this file, do not use "tabs".).
Check the glossary at the end of this file for the meaning of the keys that we will change.

Here we edit the file programmatically. These are the changes that are done
in the following:

 * adapt `MyAppCPP` and change it to `LIGURIANSEA`

 * adapt file names `VARNAME`, `GRDNAME`, `ININAME`, `BRYNAME`, `CLMNAME`, `FRCNAME` and `NFFILES` (`varinfo.yaml`, `LS2v.nc`, `ic2019.nc`, `bc2019.nc`, `clim2019.nc`, `liguriansea2019_*.nc`, `*` means the different variables). `NFFILES` is the number of forcing files.

 * change `Lm`, `Mm` and `N` based on the dimensions of your grid (make sure to read the glossary for these variable in `roms.in`)

 * read the desciption about "lateral boundary conditions" and adapt boundaries `LBC`:
    * use closed (`Clo`) for boundaries without sea-point
    * for open boundaries:
       * free-surface: Chapman implicit (`Cha`)
       * 2D U/V-momentum: Flather (`Fla`)
       * 3D U/V-momentum, temperature, salinity: Radiation with nudging (`RadNud`)
       * mixing TKE: Radiation (`Rad`)

 * set the starting time and time reference
```
DSTART = ...
TIME_REF =  18581117
```

where `DSTART` is here the number of days since 1858-11-17 or November 17, 1858 (see also [modified Julia day](https://en.wikipedia.org/wiki/Julian_day#Variants)) of the start of the model simulation (`t0` in the julia script). For instance the number of days since 2014-01-01 (year-month-day) can be computed by of following commands in Julia:

```julia
using Dates
Date(2020,1,1) - Date(1858,11,17)
```

The inverse operation can be done with:

```julia
using Dates
Date(1858,11,17) + Day(58849)
```

You can use `DateTime` if you want to specify hour, minutes or seconds.

* Adapt the length of a time step `DT` (in seconds) and number of time steps `NTIMES`
* `DT` can be 300 seconds
* Initially we choose:
    * `NTIMES` -> number of time step corresponding to 2 days (e.g. `2*24*60*60/DT` where `DT` is the time steps in seconds)
    * `NHIS`, `NAVG`-> number of time steps corresponding to 1 hour
    * `NRST` -> number of time steps correspond to 1 hour

In [ ]:
# time step (seconds)
DT = 300.
# output frequency of ROMS in time steps
NHIS = round(Int,24*60*60 / DT)
NRST = NAVG = NHIS

# number of time steps
t0 = DateTime(2023,1,1);
t1 = DateTime(2023,1,4);
NTIMES = floor(Int,Dates.value(t1-t0) / (DT * 1000))

How many CPU cores does your machine have? You can use the command `top` in a shell terminal followed by `1`.
The number of CPU cores should be `NtileI` * `NtileJ`.
The parameters `NtileI` and `NtileJ` are defined in `roms.in`.

In [ ]:
NtileI = 1
NtileJ = 1

substitutions = Dict(
    "TITLE" => "My test",
    "NtileI" => NtileI,
    "NtileJ" => NtileJ,
    "TIME_REF" => "18581117",
    "VARNAME" => var_name,
    "GRDNAME" => grd_name,
    "ININAME" => ini_name,
    "BRYNAME" => bry_name,
    "CLMNAME" => clm_name,
    "NFFILES" => length(frc_name),
    "FRCNAME" => join(frc_name,"  \\\n       "),
    "Vtransform" => domain.Vtransform,
    "Vstretching" => domain.Vstretching,
    "THETA_S" => domain.theta_s,
    "THETA_B" => domain.theta_b,
    "TCLINE" => domain.Tcline,
    "Lm" => size(domain.h,1)-2,
    "Mm" => size(domain.h,2)-2,
    "N" => domain.nlevels,
    "LBC(isFsur)" => whenopen(domain,"Cha"),
    "LBC(isUbar)" => whenopen(domain,"Fla"),
    "LBC(isVbar)" => whenopen(domain,"Fla"),
    "LBC(isUvel)" => whenopen(domain,"RadNud"),
    "LBC(isVvel)" => whenopen(domain,"RadNud"),
    "LBC(isMtke)" => whenopen(domain,"Rad"),
    "LBC(isTvar)" => whenopen(domain,"RadNud") * " \\\n" * whenopen(domain,"RadNud"),
    "DT" => DT,
    "NHIS" => NHIS,
    "NAVG" => NAVG,
    "NRST" => NRST,
    "NTIMES" => NTIMES,
    "NUDNAME" => nud_name,
    "TNUDG" => "10.0d0 10.0d0",
    "LtracerCLM" => "T T",
    "LnudgeTCLM" => "T T",
    "OBCFAC" => 10.0,
)

ROMS.infilereplace(intemplate,infile,substitutions)

Always make make sure that `THETA_S`, `THETA_B`, `TCLINE`, `Vtransform` and `Vstretching` match the values in your julia script.
We can review the changes with the shell command:
```bash
diff -u --color ~/src/roms/User/External/roms.in roms.in
```

# Run ROMS


Run ROMS with 4 CPUs splitting the domain in 2 by 2 tiles

In [ ]:
using Dates
using ROMS

Now we are ready to run the model:

In [ ]:
modeldir = expanduser("~/ROMS-implementation-test")
simulationdir = joinpath(modeldir,"Simulation1");

## Run ROMS from julia

`NtileI` and `NtileJ` must match the values in the
roms.in file.

In [ ]:
NtileI = 1
NtileJ = 1
np = NtileI*NtileJ

use_openmp = true
logfile = "roms.out"

cd(simulationdir) do
    withenv("OPAL_PREFIX" => nothing) do
        ROMS.run_model(modeldir,"roms.in"; use_openmp, np, stdout = logfile)
    end
end;

If you run into a problem, please first read the error message
carefully to get some indicaton what is wrong.

The ROMS outputs are the files `roms_his.nc` and `roms_avg.nc`.

Note that the usual method to run ROMS is from the command line.

## Run ROMS in serial

For example the serial binary `romsS` (without MPI and OpenMP) can be run as:

```bash
./romsS < roms.in | tee roms.out
```


## Run ROMS in parallel

Make sure to activate MPI or OpenMP and recompile ROMS if necessary
With MPI:

```bash
mpirun -np 4 ./romsM  roms.in | tee roms.out
```

where 4 is the number of cores to use. To use 4 CPUs, you need to set `NtileI` to 2 and `NtileJ` to 2.

With OpenMP:

```bash
OMP_NUM_THREADS=4 ./romsO < roms.out | tee roms.out
```

With the command `tee` the normal screen output will be place in the file `roms.out` but still be printed on the screen.

## Interpreting ROMS output

* Check minimum and maximum value of the different parameters
```
 NLM: GET_STATE - Read state initial conditions,             t = 57235 00:00:00
                   (Grid 02, File: roms_nest_his.nc, Rec=0182, Index=1)
                - free-surface
                   (Min = -4.63564634E-01 Max = -3.63838434E-01)
```

* The barotropic, baroclinic and Coriolis Courant number should be smaller than 1

```
 Minimum barotropic Courant Number =  2.09670689E-02
 Maximum barotropic Courant Number =  5.56799674E-01
 Maximum Coriolis   Courant Number =  1.71574766E-03
```

* Information
    * energy (kinetic, potential, total) and volume
    * maximum Courant number

```
   STEP   Day HH:MM:SS  KINETIC_ENRG   POTEN_ENRG    TOTAL_ENRG    NET_VOLUME  Grid
          C => (i,j,k)       Cu            Cv            Cw         Max Speed

 346200 57235 00:00:00  2.691184E-03  1.043099E+04  1.043099E+04  6.221264E+13  01
          (079,055,30)  9.266512E-02  4.949213E-02  0.000000E+00  1.081862E+00
```

Plot some variables like sea surface height and sea surface temperature at the beginning and the end of the simulation.
In Julia, force figure 1 and to 2 to have the same color-bar.

```julia
figure(); p1 = pcolor(randn(3,3)); colorbar()
figure(); p2 = pcolor(randn(3,3)); colorbar()
p2.set_clim(p1.get_clim())
```

* If everything runs fine,
    * is the model still stable with a longer time steps (`DT`) ?
    * increase the number of time steps (`NTIMES`)
    * possibly adapt the frequency at which you save the model results (`NHIS`, `NAVG`,`NRST`)

# Plotting ROMS results and input files

The aim here is to visualize the model files with generic plotting and analsis packages rather than to use a model specific visualization tool which hides many details and might lack of flexibility.
The necessary files are already in the directory containing the model simulation and its
parent direction (`ROMS-implementation-test`). Downloading the files is only needed if you did not run the simulation.

In [ ]:
grd_name = "roms_grd_liguriansea.nc"

if !isfile(grd_name)
    download("https://dox.ulg.ac.be/index.php/s/J9DXhUPXbyLADJa/download",grd_name)
end

fname = "roms_his.nc"
if !isfile(fname)
    download("https://dox.ulg.ac.be/index.php/s/17UWsY7tRNMDf4w/download",fname)
end

## Bathymetry

In this example, the bathymetry defined in the grid file is visualized. Make sure that your current working directory
contains the file `roms_grd_liguriansea.nc` (use e.g. `;cd ~/ROMS-implementation-test`)

In [ ]:
using ROMS, NCDatasets, GeoDatasets, Statistics
using PyPlot

ds_grid = NCDataset("roms_grd_liguriansea.nc");
lon = ds_grid["lon_rho"][:,:];
lat = ds_grid["lat_rho"][:,:];
h = nomissing(ds_grid["h"][:,:],NaN);
mask_rho = ds_grid["mask_rho"][:,:];

figure(figsize=(7,4))
hmask = copy(h)
hmask[mask_rho .== 0] .= NaN;
pcolormesh(lon,lat,hmask);
colorbar()
# or colorbar(orientation="horizontal")
gca().set_aspect(1/cosd(mean(lat)))

title("smoothed bathymetry [m]");
savefig("smoothed_bathymetry.png");

## Surface temperature

The surface surface temperature (or salinity) of the model output or
climatology file can be visualized as follows.
The parameter `n` is the time instance to plot.
Make sure that your current working directory
contains the file to plot (use e.g. `;cd ~/ROMS-implementation-test/` to plot `roms_his.nc`)

In [ ]:
# instance to plot
n = 1

ds = NCDataset("roms_his.nc")
temp = nomissing(ds["temp"][:,:,end,n],NaN);
temp[mask_rho .== 0] .= NaN;

if haskey(ds,"time")
    # for the climatology file
    time = ds["time"][:]
else
    # for ROMS output
    time = ds["ocean_time"][:]
end

figure(figsize=(7,4))
pcolormesh(lon,lat,temp)
gca().set_aspect(1/cosd(mean(lat)))
colorbar();
title("sea surface temperature [°C]")
savefig("SST.png");

Exercise:
* Plot salinity
* Plot different time instance (`n`)
* Where do we specify that the surface values are to be plotted? Plot different layers.

## Surface velocity and elevation

In [ ]:
zeta = nomissing(ds["zeta"][:,:,n],NaN)
u = nomissing(ds["u"][:,:,end,n],NaN);
v = nomissing(ds["v"][:,:,end,n],NaN);

mask_u = ds_grid["mask_u"][:,:];
mask_v = ds_grid["mask_v"][:,:];

u[mask_u .== 0] .= NaN;
v[mask_v .== 0] .= NaN;
zeta[mask_rho .== 0] .= NaN;

# ROMS uses an Arakawa C grid
u_r = cat(u[1:1,:], (u[2:end,:] .+ u[1:end-1,:])/2, u[end:end,:], dims=1);
v_r = cat(v[:,1:1], (v[:,2:end] .+ v[:,1:end-1])/2, v[:,end:end], dims=2);

# all sizes should be the same
size(u_r), size(v_r), size(mask_rho)

figure(figsize=(7,4))
pcolormesh(lon,lat,zeta)
colorbar();
# plot only a single arrow for r x r grid cells
r = 3;
i = 1:r:size(lon,1);
j = 1:r:size(lon,2);
q = quiver(lon[i,j],lat[i,j],u_r[i,j],v_r[i,j])
quiverkey(q,0.9,0.85,1,"1 m/s",coordinates="axes")
title("surface currents [m/s] and elevation [m]");
gca().set_aspect(1/cosd(mean(lat)))
savefig("surface_zeta_uv.png");

Exercise:
* The surface currents seems to follow lines of constant surface elevation. Explain why this is to be expected.

## Vertical section

In this example we will plot a vertical section by slicing the
model output at a given index.

It is very important that the parameters (`opt`) defining the vertical layer match the parameters values choosen when ROMS was setup.

In [ ]:
opt = (
    Tcline = 50,   # m
    theta_s = 5,   # surface refinement
    theta_b = 0.4, # bottom refinement
    nlevels = 32,  # number of vertical levels
    Vtransform  = 2,
    Vstretching = 4,
)

hmin = minimum(h)
hc = min(hmin,opt.Tcline)
z_r = ROMS.set_depth(opt.Vtransform, opt.Vstretching,
                   opt.theta_s, opt.theta_b, hc, opt.nlevels,
                   1, h);

temp = nomissing(ds["temp"][:,:,:,n],NaN);

mask3 = repeat(mask_rho,inner=(1,1,opt.nlevels))
lon3 = repeat(lon,inner=(1,1,opt.nlevels))
lat3 = repeat(lat,inner=(1,1,opt.nlevels))

temp[mask3 .== 0] .= NaN;

i = 20;

clf()
contourf(lat3[i,:,:],z_r[i,:,:],temp[i,:,:],40)
ylim(-300,0);
xlabel("latitude")
ylabel("depth [m]")
title("temperature at $(round(lon[i,1],sigdigits=4)) °E")
colorbar();

# inset plot
ax2 = gcf().add_axes([0.1,0.18,0.4,0.3])
ax2.pcolormesh(lon,lat,temp[:,:,end])
ax2.set_aspect(1/cosd(mean(lat)))
ax2.plot(lon[i,[1,end]],lat[i,[1,end]],"m")

savefig("temp_section1.png");

Exercise:
* Plot a section at different longitude and latitude

## Horizontal section

A horizontal at the fixed depth of 200 m is extracted and plotted.

In [ ]:
tempi = ROMS.model_interp3(lon,lat,z_r,temp,lon,lat,[-200])
mlon,mlat,mdata = GeoDatasets.landseamask(resolution='f', grid=1.25)

figure(figsize=(7,4))
pcolormesh(lon,lat,tempi[:,:,1])
colorbar();
ax = axis()
contourf(mlon,mlat,mdata',[0.5, 3],colors=["gray"])
axis(ax)
gca().set_aspect(1/cosd(mean(lat)))
title("temperature at 200 m [°C]")
savefig("temp_hsection_200.png");

## Arbitrary vertical section

The vectors `section_lon` and `section_lat` define the coordinates where we want to extract
the surface temperature.

In [ ]:
section_lon = LinRange(8.18, 8.7,100);
section_lat = LinRange(43.95, 43.53,100);

using Interpolations

function section_interp(v)
    itp = interpolate((lon[:,1],lat[1,:]),v,Gridded(Linear()))
    return itp.(section_lon,section_lat)
end

section_temp = mapslices(section_interp,temp,dims=(1,2))
section_z = mapslices(section_interp,z_r,dims=(1,2))

section_x = repeat(section_lon,inner=(1,size(temp,3)))

clf()
contourf(section_x,section_z[:,1,:],section_temp[:,1,:],50)
ylim(-500,0)
colorbar()
xlabel("longitude")
ylabel("depth")
title("temperature section [°C]");

# inset plot
ax2 = gcf().add_axes([0.4,0.2,0.4,0.3])
ax2.pcolormesh(lon,lat,temp[:,:,end])
axis("on")
ax2.set_aspect(1/cosd(mean(lat)))
ax2.plot(section_lon,section_lat,"m")

savefig("temp_vsection.png");

# Plotting ROMS results and input files

The aim here is to visualize the model files with generic plotting and analsis packages rather than to use a model specific visualization tool which hides many details and might lack of flexibility.
The necessary files are already in the directory containing the model simulation and its
parent direction (`ROMS-implementation-test`). Downloading the files is only needed if you did not run the simulation.

In [ ]:
grd_name = "roms_grd_liguriansea.nc"

if !isfile(grd_name)
    download("https://dox.ulg.ac.be/index.php/s/J9DXhUPXbyLADJa/download",grd_name)
end

fname = "roms_his.nc"
if !isfile(fname)
    download("https://dox.ulg.ac.be/index.php/s/17UWsY7tRNMDf4w/download",fname)
end

## Bathymetry

In this example, the bathymetry defined in the grid file is visualized. Make sure that your current working directory
contains the file `roms_grd_liguriansea.nc` (use e.g. `;cd ~/ROMS-implementation-test`)

In [ ]:
using ROMS, NCDatasets, GeoDatasets, Statistics
using CairoMakie # GeoMakie, GLMakie
using CairoMakie: Point2f0

ds_grid = NCDataset("roms_grd_liguriansea.nc");
lon = ds_grid["lon_rho"][:,:];
lat = ds_grid["lat_rho"][:,:];
h = ds_grid["h"][:,:]
mask_rho = ds_grid["mask_rho"][:,:];

hmask = copy(h)
hmask[mask_rho .== 0] .= missing;

fig = Figure();
ga = Axis(fig[1, 1]; title = "smoothed bathymetry [m]",
         aspect = AxisAspect(1/cosd(mean(lat))));
surf = surface!(ga,lon,lat,hmask, shading = NoShading, interpolate = false);
Colorbar(fig[1,2],surf)
xlims!(ga,extrema(lon))
ylims!(ga,extrema(lat))
save("smoothed_bathymetry_makie.png",fig); nothing # hide

## Surface temperature

The surface surface temperature (or salinity) of the model output or
climatology file can be visualized as follows.
The parameter `n` is the time instance to plot.
Make sure that your current working directory
contains the file to plot (use e.g. `;cd ~/ROMS-implementation-test/` to plot `roms_his.nc`)

In [ ]:
# instance to plot
n = 1

ds = NCDataset("roms_his.nc")
temp = ds["temp"][:,:,end,n]
temp[mask_rho .== 0] .= missing;

if haskey(ds,"time")
    # for the climatology file
    time = ds["time"][:]
else
    # for ROMS output
    time = ds["ocean_time"][:]
end

fig = Figure()
ga = Axis(fig[1, 1]; title = "sea surface temperature [degree C]")
surf = surface!(ga,lon,lat,temp, shading = NoShading, interpolate = false);
Colorbar(fig[1,2],surf);
xlims!(ga,extrema(lon))
ylims!(ga,extrema(lat))
save("SST_makie.png",fig); nothing # hide

Exercise:
* Plot salinity
* Plot different time instance (`n`)
* Where do we specify that the surface values are to be plotted? Plot different layers.

## Surface velocity and elevation

In [ ]:
zeta = nomissing(ds["zeta"][:,:,n],NaN)
u = nomissing(ds["u"][:,:,end,n],NaN);
v = nomissing(ds["v"][:,:,end,n],NaN);

mask_u = ds_grid["mask_u"][:,:];
mask_v = ds_grid["mask_v"][:,:];

u[mask_u .== 0] .= NaN;
v[mask_v .== 0] .= NaN;
zeta[mask_rho .== 0] .= NaN;

# ROMS uses an Arakawa C grid
u_r = cat(u[1:1,:], (u[2:end,:] .+ u[1:end-1,:])/2, u[end:end,:], dims=1);
v_r = cat(v[:,1:1], (v[:,2:end] .+ v[:,1:end-1])/2, v[:,end:end], dims=2);

# all sizes should be the same
size(u_r), size(v_r), size(mask_rho)

fig = Figure();
ga = Axis(fig[1, 1]; title = "surface currents [m/s] and elevation [m]",
         aspect = AxisAspect(1/cosd(mean(lat))));
surf = surface!(ga,lon,lat,zeta, shading = NoShading, interpolate = false);
Colorbar(fig[1,2],surf);
# plot only a single arrow for r x r grid cells
r = 3;
i = 1:r:size(lon,1);
j = 1:r:size(lon,2);
s = 0.6;
arrows!(ga,lon[i,j][:],lat[i,j][:],s*u_r[i,j][:],s*v_r[i,j][:]);
i=j=5;
arrows!(ga,[11],[44],[s*1],[0]);
text!(ga,[11],[44],text="1 m/s")
xlims!(ga,extrema(lon))
ylims!(ga,extrema(lat))
save("surface_zeta_uv_makie.png",fig); nothing # hide

Exercise:
* The surface currents seems to follow lines of constant surface elevation. Explain why this is to be expected.

## Vertical section

In this example we will plot a vertical section by slicing the
model output at a given index.

It is very important that the parameters (`opt`) defining the vertical layer match the parameters values choosen when ROMS was setup.

In [ ]:
opt = (
    Tcline = 50,   # m
    theta_s = 5,   # surface refinement
    theta_b = 0.4, # bottom refinement
    nlevels = 32,  # number of vertical levels
    Vtransform  = 2,
    Vstretching = 4,
)

hmin = minimum(h)
hc = min(hmin,opt.Tcline)
z_r = ROMS.set_depth(opt.Vtransform, opt.Vstretching,
                   opt.theta_s, opt.theta_b, hc, opt.nlevels,
                   1, h);

temp = nomissing(ds["temp"][:,:,:,n],NaN);

mask3 = repeat(mask_rho,inner=(1,1,opt.nlevels))
lon3 = repeat(lon,inner=(1,1,opt.nlevels))
lat3 = repeat(lat,inner=(1,1,opt.nlevels))

temp[mask3 .== 0] .= NaN;

i = 20;

fig = Figure();
ga = Axis(fig[1, 1]; title = "temperature at $(round(lon[i,1],sigdigits=4)) °E",
          xlabel = "latitude",
          ylabel = "depth [m]",
          backgroundcolor=:white,
          );
surf = surface!(ga,lat3[i,:,:],z_r[i,:,:],temp[i,:,:],shading = NoShading, interpolate = false);
xlims!(ga,extrema(lat3[i,:,:]))
ylims!(ga,-300,0);
Colorbar(fig[1,2],surf);
ax2 = Axis(
    fig[1, 1],
    width = Relative(0.4),
    height = Relative(0.3),
    halign = 0.13,
    valign = 0.18,
    aspect = AxisAspect(1/cosd(mean(lat))),
    backgroundcolor=:white);
# inset plot
poly!(ax2,Point2f0[(lon[1,1], lat[1,1]), (lon[1,1], lat[1,end]), (lon[end,1], lat[1,end]), (lon[end,1], lat[1,1])], color = [:white, :white, :white, :white])
surf = surface!(ax2,lon[:,1],lat[1,:],temp[:,:,end], shading = NoShading, interpolate = false);
#ax2.pcolormesh(lon,lat,temp[:,:,end])
#ax2.set_aspect(1/cosd(mean(lat)))
lines!(ax2,lon[i,[1,end]],lat[i,[1,end]],color="magenta")
xlims!(ax2,extrema(lon))
ylims!(ax2,extrema(lat))

save("temp_section1_makie.png",fig);

Exercise:
* Plot a section at different longitude and latitude

## Horizontal section

A horizontal at the fixed depth of 200 m is extracted and plotted.

In [ ]:
tempi = ROMS.model_interp3(lon,lat,z_r,temp,lon,lat,[-200])
mlon,mlat,mdata = GeoDatasets.landseamask(resolution='f', grid=1.25)

ii = findall(minimum(lon) .<=  mlon .<= maximum(lon))
jj = findall(minimum(lat) .<=  mlat .<= maximum(lat))

mlon = mlon[ii]
mlat = mlat[jj]
mdata = mdata[ii,jj]

fig = Figure();
ga = Axis(fig[1, 1]; title = "temperature at 200 m [°C]",
          xlabel = "longitude",
          ylabel = "latitude",
          );
surf = surface!(ga,lon,lat,tempi[:,:,1],shading = NoShading, interpolate = false);
Colorbar(fig[1,2],surf);
contourf!(ga,mlon,mlat,mdata,levels=[0.5, 3],colormap=[:grey])
xlims!(ga,extrema(lon))
ylims!(ga,extrema(lat))
fig

save("temp_hsection_200_makie.png",fig);

## Arbitrary vertical section

The vectors `section_lon` and `section_lat` define the coordinates where we want to extract
the surface temperature.

In [ ]:
section_lon = LinRange(8.18, 8.7,100);
section_lat = LinRange(43.95, 43.53,100);

using Interpolations

function section_interp(v)
    itp = interpolate((lon[:,1],lat[1,:]),v,Gridded(Linear()))
    return itp.(section_lon,section_lat)
end

section_temp = mapslices(section_interp,temp,dims=(1,2))
section_z = mapslices(section_interp,z_r,dims=(1,2))
section_x = repeat(section_lon,inner=(1,size(temp,3)))


fig = Figure();
ga = Axis(fig[1, 1]; title = "temperature section [°C]",
          xlabel = "longitude",
          ylabel = "depth",
          );
surf = surface!(ga,section_x,section_z[:,1,:],section_temp[:,1,:],shading = NoShading, interpolate = false)
ylims!(ga,-500,0)
Colorbar(fig[1,2],surf);
# inset plot
ax2 = Axis(
    fig[1, 1],
    width = Relative(0.4),
    height = Relative(0.3),
    halign = 0.6,
    valign = 0.18,
    aspect = AxisAspect(1/cosd(mean(lat))),
    backgroundcolor=:white);
poly!(ax2,Point2f0[(lon[1,1], lat[1,1]), (lon[1,1], lat[1,end]), (lon[end,1], lat[1,end]), (lon[end,1], lat[1,1])], color = [:white, :white, :white, :white])
surf = surface!(ax2,lon[:,1],lat[1,:],temp[:,:,end], shading = NoShading, interpolate = false);
xlims!(ax2,extrema(lon))
ylims!(ax2,extrema(lat))
fig

save("temp_vsection_makie.png",fig);

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*